В домашнем задании нужно решить задачу классификации типа стекол. Целевая переменная – тип стекла «Type». Остальные признаки описывают химические элементы в составе материала. Датасет нужно исследовать на наличие выбросов, провести EDA.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import DBSCAN

1. Получите данные и загрузите их в рабочую среду (Jupyter Notebook или другую).

Type of glass: (class attribute)
* building_windows_float_processed
* building_windows_non_float_processed
* vehicle_windows_float_processed
* vehicle_windows_non_float_processed (none in this database)
* containers
* tableware
* headlamps

In [ ]:
data = pd.read_csv('glass.csv')
data.head()

2. Проведите первичный анализ.
* Проверьте количество записей для каждого класса. Сделайте вывод.

In [ ]:
data['Fe'].unique()

In [ ]:
data.info()

3. Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [ ]:
data.columns

In [ ]:
X = data[['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']]
y = data['Type']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

4. Обучите модель дерева решений RandomForestClassifier на обучающем множестве.

In [ ]:
tree = RandomForestClassifier(random_state=42)
tree.fit(X_train, y_train)

5. Для тестового множества предскажите тип стекла и сравните с истинным значением, посчитав точность предсказания модели (accuracy).

In [ ]:
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)

In [ ]:
# Переобучение

accuracy = accuracy_score(y_train, y_train_pred)
print(f'Accuracy train: {accuracy:.4f}')
accuracy = accuracy_score(y_test, y_test_pred)
print(f'Accuracy test: {accuracy:.4f}')

In [ ]:
# Подберем параметры с помощью GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}

grid_search = GridSearchCV(
    estimator=tree,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print(f'Лучшие параметры: {grid_search.best_params_}')
print(f'Accuracy: {grid_search.best_score_:.2f}')

best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_train, y_train_pred)
print(f'Accuracy train: {accuracy:.4f}')
accuracy = accuracy_score(y_test, y_test_pred)
print(f'Accuracy test: {accuracy:.4f}')

6. Обработайте выбросы в данных.
* Визуализируйте распределение значений для каждой переменной. Можно использовать функции sns.boxplot, sns.distplot. Есть ли признаки с нормальным распределением?
* Исследуйте признаки на выбросы несколькими способами.
* Удалите выбросы. *Посчитайте процент удаленных записей от общего числа записей для каждого класса.

In [ ]:
data.head(2)

In [ ]:
# Согласно графику ни один из признаков не распределен нормально
                               
for feature in list(data.columns):
    plt.figure(figsize=(8,4))
    plt.hist(data[feature], bins=25)
    plt.title(f'Гистограмма распределения {feature}')
    plt.show()

In [ ]:
# Визуализируем boxplot для всех признаков 

for feature in list(data.columns):
    plt.figure(figsize=(8,4))
    plt.boxplot(data[feature], vert=False)
    plt.title(f'Boxplot по признаку {feature}')
    plt.show()

In [ ]:
# Удалим выбросы
all_outliers = []

for feature in list(data.drop('Type',axis=1).columns):
    Q1 = data[feature].quantile(0.25)
    Q3 = data[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = data[(data[feature] < lower_bound) | (data[feature] > upper_bound)]
    all_outliers.extend(outliers.index)

    print(f'Признак: {feature}')
    print(f'Выбросы: {len(outliers)}')
    print(f'Границы: {lower_bound:.3f}, {upper_bound:.3f}')

In [ ]:
outliers_data = data.loc[all_outliers]
outliers_data.head()

In [ ]:
outliers_data['RI'].sort_values().unique()

In [ ]:
set_outliers = set(all_outliers)
print(f'Количество выбросов: {len(set_outliers)}')
print(f'Процент выбросов: {len(set_outliers) / len(data) * 100:.2f}%')

In [ ]:
# Количество выбросов довольно большое

data_cleaned = data.drop(set_outliers)
print(f'Строк после удаления: {len(data_cleaned)} строк')

In [ ]:
data_cleaned['RI'].sort_values().unique()

7. Повторите п. 4, п. 5.

In [ ]:
X = data_cleaned[['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']]
y = data_cleaned['Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_train, y_train_pred)
print(f'Accuracy train: {accuracy:.4f}')
accuracy = accuracy_score(y_test, y_test_pred)
print(f'Accuracy test: {accuracy:.4f}')

In [ ]:
# Определим выбросы с помощью DBSCAN

X = data[['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

dbscan = DBSCAN(eps=2, min_samples=10)  
clusters = dbscan.fit_predict(X_scaled)

data['is_outlier'] = (clusters == -1).astype(int)

# Количество выбросов
outliers = data[data['is_outlier'] == 1]

print(f'Количество выбросов: {len(outliers)}')
print(f'Процент выбросов: {len(outliers) / len(data) * 100:.2f}%')

In [ ]:
# Удалим выбросы

data_dbscan = data[data['is_outlier'] == 0]
print(f'Строк после удаления: {len(data_dbscan)} строк')

In [ ]:
X = data_dbscan[['RI', 'Na', 'Mg', 'Al', 'Si', 'K', 'Ca', 'Ba', 'Fe']]
y = data_dbscan['Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_train, y_train_pred)
print(f'Accuracy train: {accuracy:.4f}')
accuracy = accuracy_score(y_test, y_test_pred)
print(f'Accuracy test: {accuracy:.4f}')

8. Сформулируйте выводы по проделанной работе.
* Кратко опишите, какие преобразования были сделаны с данными.
* Сравните точность двух моделей.
* Напишите свое мнение, нужно ли исследовать данные на выбросы, для чего это делается, плюсы и минусы подхода.

### Обучили модель дерева решений на исходном датасете, получили accuracy - 79%. Далее обучили модель без учета выбросов, повысили показатель метрики до 93%, но выбросами убрали 36% данных. С помощью DBSCAN убрали 14% выбросных данных, метрика 89%.